In [11]:
# Import important libraries

import numpy as np
import sympy as sp
import scipy.integrate as integrate

In [12]:
a1 = 1.5
a2 = 1
a3 = 0.5
a = np.array([a1, a2, a3])

E = 210
nu = 0.3
E_star = 210
nu_star = 0.3

eps11 = 0.001
eps22 = 0.001
eps33 = 0.001
eps12 = 0
eps13 = 0
eps23 = 0
eig_strain = np.array([[eps11,eps12,eps13],[eps12,eps22,eps23],[eps13,eps23,eps33]])

sig11 = 0
sig22 = 0
sig33 = 0
sig12 = 0
sig13 = 0
sig23 = 0
app_stress = np.array([[sig11,sig12,sig13],[sig12,sig22,sig23],[sig13,sig23,sig33]])

eps_11 = sig11/E_star - nu_star*(sig22+sig33)/E_star
eps_22 = sig22/E_star - nu_star*(sig11+sig33)/E_star
eps_33 = sig33/E_star - nu_star*(sig11+sig22)/E_star
eps_12 = sig12*(1+nu_star)/E_star
eps_13 = sig13*(1+nu_star)/E_star
eps_23 = sig23*(1+nu_star)/E_star
app_strain = np.array([[eps_11,eps_12,eps_13],[eps_12,eps_22,eps_23],[eps_13,eps_23,eps_33]])

delta = np.identity(3)

In [13]:
# Defining elliptic integrals and solving them. (page 85 of Mura's book, equation 11.36) lambda=0.. verified for lambda = 0
def I_(x,a, lambda_):
    #lambda_ = get_lambda(x,a)
    def integrand(s):
        return 1/np.sqrt((a[0]**2 + s)*(a[1]**2 + s)*(a[2]**2 + s))
    inte = integrate.quad(integrand, lambda_, np.inf)[0]
    return inte*np.pi*2*a[0]*a[1]*a[2]

def Ii_(x,a, lambda_):
    #lambda_ = get_lambda(x,a)
    arr = np.zeros(3)
    for i in range(3):
        def integrand(s):
            return (1/(np.sqrt((a[0]**2 + s)*(a[1]**2 + s)*(a[2]**2 + s))*(a[i]**2 + s)))
        inte = integrate.quad(integrand, lambda_, np.inf)[0]
        arr[i] = inte*np.pi*2*a[0]*a[1]*a[2]
    return arr

def Iij_(x,a, lambda_):
    #lambda_ = get_lambda(x,a)
    #print("lambda = ",lamb)
    arr = np.zeros((3,3))
    for i in range(3):
        for j in range(3):
            def integrand(s):
                return (1/(np.sqrt((a[0]**2 + s)*(a[1]**2 + s)*(a[2]**2 + s))*(a[i]**2 + s)*(a[j]**2 + s)))
            inte = integrate.quad(integrand, lambda_, np.inf)[0]
            arr[i,j] = inte*np.pi*2*a[0]*a[1]*a[2]
    return arr

In [14]:
# Compute Sijkl (page 88 of Mura's book, equation 11.42)

def get_Sijkl(x,a,I,Ii,Iij):
    Sijkl = np.zeros((3,3,3,3))
    for i in range(3):
        for j in range(3):
            for k in range(3):
                for l in range(3):
                    t1 = delta[i,j]*delta[k,l]*(2*nu*Ii[i] - Ii[k] + (a[i]**2)*Iij[k,i])
                    t2 = (delta[i,k]*delta[j,l] + delta[i,l]*delta[j,k])*((a[i]**2)*Iij[i,j] - Ii[j] + (1-nu)*(Ii[k] + Ii[l]))
                    Sijkl[i,j,k,l] = (t1+t2)/(8*np.pi*(1-nu))
                    #print("S",i+1,j+1,k+1,l+1," = ",Sijkl[i,j,k,l])
    return Sijkl

In [15]:
lambda_ = 0
x = 0
I = I_(x,a,lambda_)
Ii = Ii_(x,a,lambda_)
Iij = Iij_(x,a,lambda_)
S = get_Sijkl(x,a,I,Ii,Iij)

In [16]:
print(S)

[[[[ 0.27872081  0.          0.        ]
   [ 0.          0.01868742  0.        ]
   [ 0.          0.         -0.00713551]]

  [[ 0.          0.16342878  0.        ]
   [ 0.16342878  0.          0.        ]
   [ 0.          0.          0.        ]]

  [[ 0.          0.          0.29230146]
   [ 0.          0.          0.        ]
   [ 0.29230146  0.          0.        ]]]


 [[[ 0.          0.16342878  0.        ]
   [ 0.16342878  0.          0.        ]
   [ 0.          0.          0.        ]]

  [[ 0.066196    0.          0.        ]
   [ 0.          0.43261237  0.        ]
   [ 0.          0.         -0.00266515]]

  [[ 0.          0.          0.        ]
   [ 0.          0.          0.30468991]
   [ 0.          0.30468991  0.        ]]]


 [[[ 0.          0.          0.29230146]
   [ 0.          0.          0.        ]
   [ 0.29230146  0.          0.        ]]

  [[ 0.          0.          0.        ]
   [ 0.          0.          0.30468991]
   [ 0.          0.30468991  0.        

In [17]:
# Calculate meu
meu = E/(2*(1+nu))
meu_star = E_star/(2*(1+nu_star))

# calculate K and K_star and t
K = E/(3*(1-2*nu))
K_star = E_star/(3*(1-2*nu_star))
t = K_star/K

In [18]:
# Calculate shear strains
strain11 = 0
strain22 = 0
strain33 = 0
strain12 = (2*(meu-meu_star)*eps_12 + 2*meu_star*eps12)/(4*(meu_star-meu)*S[0,1,0,1] + 2*meu)
strain13 = (2*(meu-meu_star)*eps_13 + 2*meu_star*eps13)/(4*(meu_star-meu)*S[0,2,0,2] + 2*meu)
strain23 = (2*(meu-meu_star)*eps_23 + 2*meu_star*eps23)/(4*(meu_star-meu)*S[1,2,1,2] + 2*meu)
print("strain12 = ",strain12)
print("strain13 = ",strain13)
print("strain23 = ",strain23)

strains = np.array([[strain11,strain12,strain13],[strain12,strain22,strain23],[strain13,strain23,strain33]])


strain12 =  0.0
strain13 =  0.0
strain23 =  0.0


In [19]:
# Formulate a system of equations to solve for strain11, strain22 and strain33

a = np.zeros((3,3))
b = np.zeros(3)

a[0,0] = (1-t)*S[0,0,0,0] - 1
a[0,1] = (1-t)*S[0,0,1,1]
a[0,2] = (1-t)*S[0,0,2,2]
a[1,0] = (1-t)*S[1,1,0,0]
a[1,1] = (1-t)*S[1,1,1,1] - 1
a[1,2] = (1-t)*S[1,1,2,2]
a[2,0] = (1-t)*S[2,2,0,0]
a[2,1] = (1-t)*S[2,2,1,1]
a[2,2] = (1-t)*S[2,2,2,2] - 1

t1 = (t-1)*eps_11 - t*eps11
t2 = 0
for m in range(3):
    for n in range(3):
        if m != n:
            t2 += (1-t)*S[0,0,m,n]*strains[m,n]
b[0] = t1 - t2

t1 = (t-1)*eps_22 - t*eps22
t2 = 0
for n in range(3):
    if m != n:
        t2 += (1-t)*S[1,1,m,n]*strains[m,n]
b[1] = t1 - t2

t1 = (t-1)*eps_33 - t*eps33
t2 = 0
for m in range(3):
    for n in range(3):
        if m != n:
            t2 += (1-t)*S[2,2,m,n]*strains[m,n]
b[2] = t1 - t2

print("a = ",a)
print("b = ",b)

x = np.linalg.solve(a,b)
print("strain11 = ",x[0])
print("strain22 = ",x[1])
print("strain33 = ",x[2])

a =  [[-1.  0. -0.]
 [ 0. -1. -0.]
 [ 0.  0. -1.]]
b =  [-0.001 -0.001 -0.001]
strain11 =  0.001
strain22 =  0.001
strain33 =  0.001


In [20]:
strain11 = x[0]
strain22 = x[1]
strain33 = x[2]
strains = np.array([[strain11,strain12,strain13],[strain12,strain22,strain23],[strain13,strain23,strain33]])
print("strains = ",strains)

strains =  [[0.001 0.    0.   ]
 [0.    0.001 0.   ]
 [0.    0.    0.001]]


In [21]:
# Stress inside the inclusion
ld = 2*meu*nu/(1-2*nu)
#hydrostatic term
hterm = 0
for k in range(3):
    for m in range(3):
        for n in range(3):
            hterm += S[k,k,m,n]*strains[m,n]
hterm = hterm + eps_11 + eps_22 + eps_33 - strain11 - strain22 - strain33

suma = 0
for m in range(3):
    for n in range(3):
        suma = suma + S[0,0,m,n]*strains[m,n]
sig_11 = 2*meu*(eps_11 - strain11 + suma) + ld*hterm

suma = 0
for m in range(3):
    for n in range(3):
        suma = suma + S[1,1,m,n]*strains[m,n]
sig_22 = 2*meu*(eps_22 - strain22 + suma) + ld*hterm

suma = 0
for m in range(3):
    for n in range(3):
        suma = suma + S[2,2,m,n]*strains[m,n]
sig_33 = 2*meu*(eps_33 - strain33 + suma) + ld*hterm

suma = 0
for m in range(3):
    for n in range(3):
        suma = suma + S[0,1,m,n]*strains[m,n]
sig_12 = 2*meu*(eps_12 - strain12 + suma)

suma = 0
for m in range(3):
    for n in range(3):
        suma = suma + S[0,2,m,n]*strains[m,n]
sig_13 = 2*meu*(eps_13 - strain13 + suma)

suma = 0
for m in range(3):
    for n in range(3):
        suma = suma + S[1,2,m,n]*strains[m,n]
sig_23 = 2*meu*(eps_23 - strain23 + suma)

total_sigma = np.array([[sig_11,sig_12,sig_13],[sig_12,sig_22,sig_23],[sig_13,sig_23,sig_33]])
print("total_sigma = ",total_sigma)


total_sigma =  [[-0.25310979  0.          0.        ]
 [ 0.         -0.21985379  0.        ]
 [ 0.          0.         -0.12703642]]
